<a href="https://www.kaggle.com/code/trakiet/duachuot?scriptVersionId=269267411" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


/kaggle/input/mooccubex-dataset/course-teacher.txt
/kaggle/input/mooccubex-dataset/course.json
/kaggle/input/mooccubex-dataset/paper.json
/kaggle/input/mooccubex-dataset/reply.json
/kaggle/input/mooccubex-dataset/concept-comment.txt
/kaggle/input/mooccubex-dataset/user-video.json
/kaggle/input/mooccubex-dataset/course-comment.txt
/kaggle/input/mooccubex-dataset/concept-problem.txt
/kaggle/input/mooccubex-dataset/user-reply.txt
/kaggle/input/mooccubex-dataset/school.json
/kaggle/input/mooccubex-dataset/translated_reply.json
/kaggle/input/mooccubex-dataset/video.json
/kaggle/input/mooccubex-dataset/concept-video.txt
/kaggle/input/mooccubex-dataset/other.json
/kaggle/input/mooccubex-dataset/course-school.txt
/kaggle/input/mooccubex-dataset/problem.json
/kaggle/input/mooccubex-dataset/user-comment.txt
/kaggle/input/mooccubex-dataset/user.json
/kaggle/input/mooccubex-dataset/exercise-problem.txt
/kaggle/input/mooccubex-dataset/concept-paper.txt
/kaggle/input/mooccubex-dataset/translated_com

In [23]:
BASE_DIR = '/kaggle/input/mooccubex-dataset/'
df_user_course_path = '/kaggle/input/user-course-v4/user_course_v4.csv'
df_use_video_final_path = '/kaggle/input/df-user-video-final/part-00000-8eb8369b-ec66-434a-9e3c-608414a06b1f-c000.csv'

In [3]:
import pyspark
from pyspark.sql import SparkSession
import warnings
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType
import seaborn as sns
import matplotlib.pyplot as plt
from os import truncate
from pyspark.sql.functions import explode, col, sum, from_unixtime, hour, dayofweek, count, round, when, isnull, split, avg
warnings.filterwarnings("ignore")

spark = SparkSession.builder \
  .appName("reply_user_video_analysis") \
  .master("local[*]") \
  .config("spark.some.config.option", "some-value") \
  .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/20 01:04:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
!ls {BASE_DIR}

comment-reply.txt    course-school.txt	      translated_reply.json
concept-comment.txt  course-teacher.txt       user-comment.txt
concept.json	     exercise-problem.txt     user.json
concept-other.txt    other.json		      user-problem.json
concept-paper.txt    paper.json		      user-reply.txt
concept-problem.txt  problem.json	      user-video.json
concept-reply.json   reply.json		      user-xiaomu.json
concept-video.txt    reply-reply.txt	      video_id-ccid.txt
course-comment.txt   school.json	      video.json
course-field.json    teacher.json
course.json	     translated_comment.json


In [24]:
df_course = spark.read.json(BASE_DIR + '/course.json')

In [25]:
df_video_course_map = (
    df_course
    .withColumn(
        "video_resource",
        F.filter(F.col("resource"), lambda x: x["resource_id"].startswith("V_"))
    )
    .withColumn("video_struct", F.explode_outer(F.col("video_resource")))
    .select(
        F.col("id").alias("course_id"),
        F.col("video_struct.resource_id").alias("video_id")
    )
)

df_video_course_map.show()

+---------+--------+
|course_id|video_id|
+---------+--------+
| C_584313|   V_849|
| C_584313|   V_850|
| C_584313|   V_851|
| C_584313|   V_857|
| C_584313|   V_859|
| C_584313|   V_861|
| C_584313|   V_862|
| C_584313|   V_867|
| C_584313|   V_868|
| C_584313|   V_869|
| C_584313|   V_870|
| C_584313|   V_871|
| C_584313|   V_877|
| C_584313|   V_878|
| C_584313|   V_879|
| C_584313|   V_880|
| C_584313|   V_881|
| C_584313|   V_883|
| C_584313|   V_884|
| C_584313|   V_885|
+---------+--------+
only showing top 20 rows



In [27]:
df_video_course_map_clean = df_video_course_map.withColumn(
    "course_id", regexp_replace("course_id", "^C_", "").cast("long")
).withColumn(
    "video_id", regexp_replace("video_id", "^V_", "").cast("long")
)

df_video_course_map_clean.show()

+---------+--------+
|course_id|video_id|
+---------+--------+
|   584313|     849|
|   584313|     850|
|   584313|     851|
|   584313|     857|
|   584313|     859|
|   584313|     861|
|   584313|     862|
|   584313|     867|
|   584313|     868|
|   584313|     869|
|   584313|     870|
|   584313|     871|
|   584313|     877|
|   584313|     878|
|   584313|     879|
|   584313|     880|
|   584313|     881|
|   584313|     883|
|   584313|     884|
|   584313|     885|
+---------+--------+
only showing top 20 rows



In [30]:
df_video = spark.read.json(BASE_DIR + "/video.json")
df_video.show()

+--------------------+--------------------+----------------------+--------------------+------------------------------------+
|                ccid|                 end|                  name|               start|                                text|
+--------------------+--------------------+----------------------+--------------------+------------------------------------+
|0001603F826A3DDA9...|[4.255, 8.119, 10...|                 Video|[1.031, 7.095, 8....|[第二个就是短助记符在生成上面。有...|
|0003DB14A14A53279...|[12.581, 17.38, 1...|                 Video|[11.799, 12.581, ...|  [大家好, 今天我们讨论一维系统辐...|
|0004A5C6F07E369D9...|[20.83, 23.39, 26...|        家禽的消化系统|[19.2, 20.84, 23....|   [各位同学 大家好, 我是来自中国...|
|00059EBD1371A6959...|[23.14, 194.458, ...|    1.2 计算机硬件系统|[19.372, 190.522,...| [大家好 本视频介绍计算机硬件系统...|
|0005D1DC01B4EF8F9...|[25.08, 26.775, 2...|      什么是供应链网络|[24.33, 25.081, 2...|   [大家好, 今天我们学习的知识点,...|
|0007919ED7652A779...|[7.26, 9.97, 11.5...|    1.2 英语文字的起源|[6.29, 7.31, 10.0...|  [同学们好, 今天我们

In [31]:
from pyspark.sql.functions import array_max
df_video = (
    df_video
    .select(
        "ccid", 
        array_max("end").alias("video_duration")
    )
)


In [32]:
df_video.show()

+--------------------+--------------+
|                ccid|video_duration|
+--------------------+--------------+
|0001603F826A3DDA9...|       509.095|
|0003DB14A14A53279...|       367.336|
|0004A5C6F07E369D9...|       492.315|
|00059EBD1371A6959...|       630.757|
|0005D1DC01B4EF8F9...|        408.75|
|0007919ED7652A779...|        506.57|
|0007C4A5229EE3CF9...|       1190.78|
|0008C8CC056F4EAC9...|       130.309|
|000948D39EA6B0A69...|        421.32|
|0009510AE8068CBA9...|        622.68|
|000A1158B61C13B79...|       1281.36|
|000A40E1D226AEE09...|       255.465|
|000A5EE0C695E8EF9...|         553.8|
|000BD50F6730291F9...|       1328.56|
|000E08952A010F3D9...|         590.0|
|000F83F0945DC9D79...|        659.68|
|0012B03A05345F229...|         17.18|
|0012D2CEBDFED7729...|        623.04|
|00137081D7B0E4219...|        530.71|
|00137797685039299...|       351.188|
+--------------------+--------------+
only showing top 20 rows



In [34]:
df_video_id_ccid = spark.read.text(BASE_DIR + '/video_id-ccid.txt')
# Parse mapping file
df_video_id_ccid_v2 = df_video_id_ccid.select(
    split(col("value"), "\t")[0].alias("video_id"),
    split(col("value"), "\t")[1].alias("ccid")
)
df_video_id_ccid_v2.show(5, truncate=False)

+--------+--------------------------------+
|video_id|ccid                            |
+--------+--------------------------------+
|V_234845|0000363DB5B6E0869C33DC5901307461|
|V_234876|0000363DB5B6E0869C33DC5901307461|
|V_234907|0000363DB5B6E0869C33DC5901307461|
|V_293392|0000363DB5B6E0869C33DC5901307461|
|V_293445|0000363DB5B6E0869C33DC5901307461|
+--------+--------------------------------+
only showing top 5 rows



In [38]:
video_info = df_video_id_ccid_v2.join(df_video, on=['ccid'])
video_info_cleaned = video_info.withColumn(
    "video_id", regexp_replace("video_id", "^V_", "").cast("long")
)
video_info_cleaned.show(truncate=False)


+--------------------------------+--------+--------------+
|ccid                            |video_id|video_duration|
+--------------------------------+--------+--------------+
|035503AF2FB5812C9C33DC5901307461|125885  |1051.907      |
|035503AF2FB5812C9C33DC5901307461|126437  |1051.907      |
|035503AF2FB5812C9C33DC5901307461|126621  |1051.907      |
|035503AF2FB5812C9C33DC5901307461|681656  |1051.907      |
|035503AF2FB5812C9C33DC5901307461|793328  |1051.907      |
|035503AF2FB5812C9C33DC5901307461|793761  |1051.907      |
|035503AF2FB5812C9C33DC5901307461|794195  |1051.907      |
|035503AF2FB5812C9C33DC5901307461|1260909 |1051.907      |
|035503AF2FB5812C9C33DC5901307461|2357873 |1051.907      |
|035503AF2FB5812C9C33DC5901307461|2358059 |1051.907      |
|035503AF2FB5812C9C33DC5901307461|2612910 |1051.907      |
|035503AF2FB5812C9C33DC5901307461|2748021 |1051.907      |
|035503AF2FB5812C9C33DC5901307461|2748207 |1051.907      |
|035503AF2FB5812C9C33DC5901307461|2971514 |1051.907     

In [43]:
df_user_course = spark.read.csv(df_user_course_path, header=True, inferSchema=True)
df_user_course_clean = df_user_course.withColumn(
    "user_id", regexp_replace("user_id", "^U_", "").cast("long")
).withColumn(
    "course_id", regexp_replace("course_id", "^C_", "").cast("long")
)

df_user_course_clean.show()

+-------+---------+-----------+----------+----------+-------------+-------------------+----------+
|user_id|course_id|certificate|start_date|  end_date|duration_days|        enroll_time|remain_day|
+-------+---------+-----------+----------+----------+-------------+-------------------+----------+
|  10000|  2033958|        1.0|2020-09-04|2020-12-31|        118.0|2020-10-27 09:07:30|      65.0|
|1000129|  1925903|        1.0|2020-08-12|2020-12-31|        141.0|2020-09-06 14:37:17|     116.0|
|1000129|  1992970|        1.0|2020-08-27|2020-12-31|        126.0|2020-09-06 14:45:37|     116.0|
|1000129|   680884|        1.0|2020-09-01|2020-12-31|        121.0|2020-10-16 10:41:09|      76.0|
|1000342|   697791|        0.0|2020-01-20|2020-07-31|        193.0|2020-02-28 10:27:22|     154.0|
|1000663|  2328510|        0.0|2020-11-18|2021-02-28|        102.0|2020-11-27 22:41:36|      93.0|
| 100071|   696601|        1.0|2019-12-25|2020-06-30|        188.0|2020-04-15 08:51:39|      76.0|
|1000826| 

In [41]:
df_user_video_final = spark.read.csv(df_use_video_final_path, header=True, inferSchema=True)
df_user_video_final.show()

+-------+--------+-----------------+------------------+------------------+----------------------+-----------------+-------------------+------------------+
|user_id|video_id|   video_duration|       rewind_time| fast_forward_time|first_local_start_time|actual_watch_time|weighted_watch_time|     average_speed|
+-------+--------+-----------------+------------------+------------------+----------------------+-----------------+-------------------+------------------+
|    112| 1395635|            170.0|               0.0|               0.0|   2020-05-02 16:47:25|             35.0|               35.0|               1.0|
|    172| 6432959|           4556.8|               0.0|               0.0|   2020-08-23 14:23:14|4.699999999999818|  4.699999999999818|               1.0|
|    197| 1358557|          857.941|           344.241|           433.656|   2020-02-21 03:57:27|             40.0|               40.0|               1.0|
|    197| 1358708|            535.1|               0.0| 52.69999999999

In [13]:
from pyspark.sql.functions import regexp_replace, col

df_cleaned = df_flatten.withColumn(
    "user_id", regexp_replace("user_id", "^U_", "").cast("long")
).withColumn(
    "video_id", regexp_replace("video_id", "^V_", "").cast("long")
)

df_cleaned.show(5, truncate=False)


+-------+--------+-----------+---------+-----+-------------------+
|user_id|video_id|start_point|end_point|speed|local_start_time   |
+-------+--------+-----------+---------+-----+-------------------+
|112    |1395633 |130.0      |190.0    |1.0  |2020-05-02 14:52:24|
|112    |1395633 |220.0      |250.0    |1.0  |2020-05-02 14:53:54|
|112    |1395633 |478.8      |508.8    |1.0  |2020-05-02 16:35:59|
|112    |1395633 |655.1      |692.55   |1.25 |2020-05-02 16:38:34|
|112    |1395635 |135.0      |170.0    |1.0  |2020-05-02 16:47:25|
+-------+--------+-----------+---------+-----+-------------------+
only showing top 5 rows



In [20]:
# # Lưu ra file CSV
# df_final.coalesce(1).write.mode("overwrite").option("header", "true").csv("/kaggle/working/df_user_video_final_output_csv")

In [22]:
# import shutil

# input_dir = "/kaggle/working/df_user_video_final_output_csv"
# output_zip = "/kaggle/working/df_user_video_final_output_csv"

# shutil.make_archive(output_zip, 'zip', input_dir)
# print("✅ Đã nén file thành df_user_video_final_output_csv.zip")


✅ Đã nén file thành df_user_video_final_output_csv.zip
